In [25]:
import urllib, json
import pandas as pd
import re
from itertools import permutations
from pulp import *

In [44]:
col_names=["displayName", "salary", "team", "position","opponent","ceiling","floor","points"]
current = pd.read_csv('week12.csv', names=col_names, header=None)

In [45]:
current.head()


,displayName,salary,team,position,opponent,ceiling,floor,points
0,Ryan Tannehill,5300,TEN,QB,JAC,26.32973,5.75218,15.89
1,Dak Prescott,6000,DAL,QB,NEP,28.71792,7.72632,17.64
2,Nick Foles,5400,JAC,QB,TEN,26.62387,5.44544,15.47
3,Ryan Finley,4700,CIN,QB,PIT,20.17050,4.04600,11.90
4,Sam Darnold,5800,NYJ,QB,OAK,26.98230,7.44983,16.97


In [46]:
availables = current[["position", "displayName", "salary",
  "points"]].groupby(["position", "displayName", "salary",
  "points"]).agg("count")
availables = availables.reset_index()
availables[availables.position=="QB"].head(15)

,position,displayName,salary,points
306,QB,Aaron Rodgers,6000,18.80
307,QB,Baker Mayfield,5900,18.43
308,QB,Brandon Allen,4600,12.91
309,QB,Carson Wentz,5600,17.72
310,QB,Dak Prescott,6000,17.64
311,QB,Daniel Jones,5700,15.20
312,QB,Derek Carr,6100,17.08
313,QB,Deshaun Watson,6500,20.94
314,QB,Drew Brees,6600,19.33
315,QB,Dwayne Haskins,4900,13.55


In [47]:
salaries = {}
points = {}
for pos in availables.position.unique():
    available_pos = availables[availables.position == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

In [48]:
pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 3,
    "TE": 1,
    "FLEX": 1,
    "DST": 1
}

In [49]:
SALARY_CAP = 50000

In [50]:
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [51]:
prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
position_constraints = []
# Setting up the reward
for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    
prob += lpSum(rewards)
prob += lpSum(costs) <= SALARY_CAP

In [52]:
prob.solve()

1

In [53]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [54]:
summary(prob)

Variables:

DST_Miami_Dolphins = 1.0
FLEX_N'Keal_Harry = 1.0
QB_Lamar_Jackson = 1.0
RB_Alvin_Kamara = 1.0
RB_Brian_Hill = 1.0
TE_Vance_McDonald = 1.0
WR_Julio_Jones = 1.0
WR_Michael_Thomas = 1.0
WR_N'Keal_Harry = 1.0
---------------------------------------

Constraints:
2000*1.0 + 3300*1.0 + 7400*1.0 + 8200*1.0 + 4900*1.0 + 3500*1.0 + 8000*1.0 + 9300*1.0 + 3300*1.0 = 49900.0
---------------------------------------

Score:
6.71*1.0 + 11.06*1.0 + 24.13*1.0 + 22.03*1.0 + 14.13*1.0 + 9.72*1.0 + 21.16*1.0 + 24.34*1.0 + 11.06*1.0 = 144.34
